In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [3]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


df = describe_dataset("./CSV_dataset/train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

In [4]:
# load dataset
df = pd.read_csv("./CSV_dataset/train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "H", "label"] = 1
df.loc[df["label"] == "L", "label"] = 2

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 1058 
Number of columns: 69

Labels: 
0    398
1    367
2    293
Name: label, dtype: int64



In [5]:
# Standard Scaling of features
# Dump input scaler
with open("C:/Users/Alrowad/Exercise-correction/plank_model/models/input_scaler.pkl", "rb") as f2:
    sc = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [7]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [8]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")
            

def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [9]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [10]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)
tuner_3l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 02s]
accuracy: 0.8640661835670471

Best accuracy So Far: 0.991725742816925
Total elapsed time: 00h 00m 38s
INFO:tensorflow:Oracle triggered exit


In [11]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-2: 384 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-3: 3 units, func:  <function softmax at 0x000001CA7D8083A0>

Other params:
learning_rate: 0.001
Epoch 1/100
85/85 [==============================] - 1s 6ms/step - loss: 0.4413 - accuracy: 0.8534 - val_loss: 0.1690 - val_accuracy: 0.9528
Epoch 2/100
85/85 [==============================] - 0s 3ms/step - loss: 0.1360 - accuracy: 0.9574 - val_loss: 0.0931 - val_accuracy: 0.9717
Epoch 3/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0896 - accuracy: 0.9787 - val_loss: 0.0759 - val_accuracy: 0.9764
Epoch 4/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0787 - accuracy: 0.9811 - val_loss: 0.0905 - val_accuracy: 0.9670
Epoch 5/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0596 - accuracy: 0.9835 - val_loss: 0.0688 - val_accuracy: 0.9764
Epoch 6/100
85/85 [================

In [12]:
final_models["3_layers"] = model_3l

In [13]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [14]:
import keras
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_1'
)

tuner_5l.search(
    x_train, 
    y_train, 
    epochs=10, 
    callbacks=[keras.callbacks.EarlyStopping(patience=5)]
)

Trial 30 Complete [00h 00m 03s]
accuracy: 0.9728132486343384

Best accuracy So Far: 0.9952718615531921
Total elapsed time: 00h 00m 53s
INFO:tensorflow:Oracle triggered exit


In [15]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-2: 96 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-3: 416 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-4: 384 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-5: 3 units, func:  <function softmax at 0x000001CA7D8083A0>

Other params:
learning_rate: 0.001
Epoch 1/100
85/85 [==============================] - 1s 7ms/step - loss: 0.3406 - accuracy: 0.8948 - val_loss: 0.1275 - val_accuracy: 0.9717
Epoch 2/100
85/85 [==============================] - 0s 5ms/step - loss: 0.1141 - accuracy: 0.9622 - val_loss: 0.0607 - val_accuracy: 0.9858
Epoch 3/100
85/85 [==============================] - 0s 5ms/step - loss: 0.0518 - accuracy: 0.9858 - val_loss: 0.1849 - val_accuracy: 0.9151
Epoch 4/100
85/85 [==============================] - 0s 5ms/step - loss: 0.0826 - accuracy: 0.9811 - val_loss: 0.0723 - val_accuracy: 0.9858
Epoch 5/100
85/85 [============================

In [16]:
final_models["5_layers"] = model_5l

In [17]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [18]:
def get_early_stopping_callback():
    return keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3
    )

tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_4'
)
tuner_7lD.search(x_train, y_train, epochs=10, callbacks=[get_early_stopping_callback()])

Trial 30 Complete [00h 00m 05s]
accuracy: 0.9728132486343384

Best accuracy So Far: 0.9905437231063843
Total elapsed time: 00h 00m 59s
INFO:tensorflow:Oracle triggered exit


In [19]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-2: 96 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-3: 0 units, func: None
Layer-4: 32 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-7: 3 units, func:  <function softmax at 0x000001CA7D8083A0>

Other params:
dropout_1: 0.1
dropout_2: 0.1
learning_rate: 0.001
Epoch 1/100
85/85 [==============================] - 1s 5ms/step - loss: 0.4502 - accuracy: 0.8215 - val_loss: 0.1225 - val_accuracy: 0.9764
Epoch 2/100
85/85 [==============================] - 0s 3ms/step - loss: 0.1353 - accuracy: 0.9563 - val_loss: 0.0812 - val_accuracy: 0.9811
Epoch 3/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0745 - accuracy: 0.9835 - val_loss: 0.0572 - val_accuracy: 0.9764
Epoch 4/100
85/85 [==============================] - 0s 3ms/step - loss: 0.0677 - accuracy: 0.9823 - 

In [20]:
final_models["7_layers_with_dropout"] = model_7lD

In [21]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [22]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_5'
)
tuner_7l.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Trial 30 Complete [00h 00m 04s]
accuracy: 0.9834515452384949

Best accuracy So Far: 0.9929078221321106
Total elapsed time: 00h 01m 06s
INFO:tensorflow:Oracle triggered exit


In [23]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-2: 512 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-3: 512 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-4: 32 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-5: 224 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-6: 128 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-7: 3 units, func:  <function softmax at 0x000001CA7D8083A0>

Other params:
learning_rate: 0.001
Epoch 1/100
85/85 [==============================] - 1s 9ms/step - loss: 0.3936 - accuracy: 0.8522 - val_loss: 0.1055 - val_accuracy: 0.9670
Epoch 2/100
85/85 [==============================] - 1s 7ms/step - loss: 0.1419 - accuracy: 0.9610 - val_loss: 0.1257 - val_accuracy: 0.9528
Epoch 3/100
85/85 [==============================] - 1s 7ms/step - loss: 0.1132 - accuracy: 0.9657 - val_loss: 0.0744 - val_accuracy: 0.9764
Epoch 4/100
85/85 [==============================] - 1s 8m

In [24]:
final_models["7_layers"] = model_7l

In [25]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-2: 384 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-3: 3 units, func:  <function softmax at 0x000001CA7D8083A0>

5_layers: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-2: 96 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-3: 416 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-4: 384 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-5: 3 units, func:  <function softmax at 0x000001CA7D8083A0>

7_layers_with_dropout: Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x000001CA7D808DC0>
Layer-2: 96 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-3: 0 units, func: None
Layer-4: 32 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function tanh at 0x000001CA7D80E160>
Layer-7: 3 units, func:  <function

In [26]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers,"[0.974, 0.988, 1.0]","[0.987, 1.0, 0.964]","[0.98, 0.994, 0.982]","[[75, 1, 0], [0, 80, 0], [2, 0, 54]]"
1,7_layers_with_dropout,"[0.962, 0.988, 1.0]","[0.987, 0.988, 0.964]","[0.974, 0.988, 0.982]","[[75, 1, 0], [1, 79, 0], [2, 0, 54]]"
2,3_layers,"[0.949, 0.988, 1.0]","[0.987, 0.988, 0.946]","[0.968, 0.988, 0.972]","[[75, 1, 0], [1, 79, 0], [3, 0, 53]]"
3,5_layers,"[0.926, 1.0, 0.981]","[0.987, 0.988, 0.911]","[0.955, 0.994, 0.944]","[[75, 0, 1], [1, 79, 0], [5, 0, 51]]"


In [27]:
# load dataset
test_df = pd.read_csv("./CSV_dataset/test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "H", "label"] = 1
test_df.loc[test_df["label"] == "L", "label"] = 2

print(f'Number of rows: {test_df.shape[0]} \nNumber of columns: {test_df.shape[1]}\n')
print(f"Labels: \n{test_df['label'].value_counts()}\n")

Number of rows: 419 
Number of columns: 69

Labels: 
1    165
0    139
2    115
Name: label, dtype: int64



In [28]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [29]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[0.951, 0.982, 0.991]","[0.978, 0.976, 0.965]","[0.965, 0.979, 0.978]","[[136, 2, 1], [4, 161, 0], [3, 1, 111]]"
1,3_layers,"[0.938, 0.981, 0.991]","[0.978, 0.964, 0.965]","[0.958, 0.972, 0.978]","[[136, 2, 1], [6, 159, 0], [3, 1, 111]]"
2,7_layers,"[0.924, 0.964, 0.991]","[0.957, 0.97, 0.939]","[0.94, 0.967, 0.964]","[[133, 5, 1], [5, 160, 0], [6, 1, 108]]"
3,5_layers,"[0.889, 0.987, 0.981]","[0.978, 0.952, 0.913]","[0.932, 0.969, 0.946]","[[136, 1, 2], [8, 157, 0], [9, 1, 105]]"


In [30]:
final_models["7_layers_with_dropout"].save("C:/Users/Alrowad/Exercise-correction/plank_model/models/plank_dp.h5")

In [31]:
for model_name, model in final_models.items():
    model.save(f"C:/Users/Alrowad/Exercise-correction/plank_model/models/{model_name}.h5")